# Job Interview Wihth OpenAI's GPT model


# Define the interview parameters


In [ ]:
jobTitle = "Cloud Engineer"
numberOfQuestions = 5


# Install libraries


In [ ]:
! pip3 install - -upgrade pip
! pip3 install azure-cognitiveservices-speech
! pip3 install python-dotenv
! pip3 install openai


# Initialize


In [ ]:
from dotenv import load_dotenv
import re
import os
import openai


load_dotenv(override=True)

settings = {
    'speechKey': os.environ.get('SPEECH_KEY'),
    'region': os.environ.get('SPEECH_REGION'),
    # Feel free to hardcode the language
    'language': os.environ.get('SPEECH_LANGUAGE'),
    'openAIKey': os.environ.get('OPENAI_KEY')
}
# Load your API key from an environment variable or secret management service
openai.api_key = settings['openAIKey']


# Utility functions


In [ ]:
def complete_openai(prompt, token=50):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        temperature=0.9,
        max_tokens=token,
        top_p=1
    )
    lines = response.to_dict_recursive()['choices'][0]['text'].split("\n")
    response = list(filter(lambda x: x != '', lines))
    return response

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import time

prop = False


def Start_recording_answer():
    # print("===================================")
    # print("Processing "+settings['fileName'])
    # print("===================================")

    # Creates an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(
        subscription=settings['speechKey'], region=settings['region'])

    speech_config.request_word_level_timestamps()
    speech_config.set_property(
        property_id=speechsdk.PropertyId.SpeechServiceResponse_OutputFormatOption, value="detailed")

    # Creates a speech recognizer using file as audio input.
    # device_name="BuiltInMicrophoneDevice")
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)

    # audio_input = speechsdk.audio.AudioConfig(filename=settings['fileName'])

    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config, audio_config=audio_config)

    # speech_recognizer = speechsdk.SpeechRecognizer(
    #     speech_config=speech_config, language=settings['language'], audio_config=audio_input)
    # translator = speechsdk.translation.TranslationRecognizer(
    #     translation_config=translation_config, audio_config=audio_input)

    # initialize some variables
    results = []
    done = False

    # Event handler to add event to the result list

    def handleResult(evt):
        import json
        nonlocal results
        results.append(json.loads(evt.result.json))

        # print the result (optional, otherwise it can run for a few minutes without output)
        # print('RECOGNIZED: {}'.format(evt))

        # result object
        res = {'text': evt.result.test, 'timestamp': evt.result.offset,
               'duration': evt.result.duration, 'raw': evt.result}

        if (evt.result.text != ""):
            results.append(res)
            # print(evt.result)

    # Event handler to check if the recognizer is done

    def stop_cb(evt):
        # print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    # Connect callbacks to the events fired by the speech recognizer & displays the info/status
    # Ref:https://docs.microsoft.com/en-us/python/api/azure-cognitiveservices-speech/azure.cognitiveservices.speech.eventsignal?view=azure-python
    # speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    # speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.session_started.connect(
        lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(
        lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(
        lambda evt: print('CANCELED {}'.format(evt)))
    speech_recognizer.recognized.connect(handleResult)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Starts continuous speech recognition
    # speech_recognizer.start_continuous_recognition()

    # Starts continuous speech recognition
    # translator.start_continuous_recognition()

    result_future = speech_recognizer.start_continuous_recognition_async()
    result_future.get()
    # Wait for speech recognition to complete
    while not done:
        time.sleep(1)
        print('type "stop" then enter when done')
        stop = input()
        if (stop.lower() == "stop"):
            print('Stopping async recognition.')
            speech_recognizer.stop_continuous_recognition_async()
            while not done:
                time.sleep(1)

    output = ""
    for res in results:
        output += res['NBest'][0]['Display']

    return results


In [ ]:
import azure.cognitiveservices.speech as speechsdk


def speak(text):
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(
        subscription=settings['speechKey'], region=settings['region'])
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

    # The language of the voice that speaks.
    speech_config.speech_synthesis_voice_name = 'en-US-JennyNeural'

    speech_synthesizer = speechsdk.SpeechSynthesizer(
        speech_config=speech_config, audio_config=audio_config)

    speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()

    if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized for text [{}]".format(text))
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print("Speech synthesis canceled: {}".format(
            cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(
                    cancellation_details.error_details))
                print("Did you set the speech resource key and region values?")


# Interview


In [ ]:
dialog = []


def get_feedback(interviewer, interviewee):
    prompt = """
write a paragraph to the interviewee describing what they did well in their response, and how they can improve:

interviewer: {0}

interviewee: {1}
""".format(interviewer, interviewee)
    print(prompt)
    res = complete_openai(prompt=prompt, token=int(4000-len(prompt.split())*1.75))
    res = ("\n".join(res))
    return res


In [ ]:
intro = complete_openai(
    prompt="Write the very first dialog for an interviewer interviewing an interviewee applying for a {0} job. Don't write any of the interviewee parts, and only write 1 part for the interviewer".format(jobTitle), token=3000)
introDialog = ("\n".join(intro)).replace("Interviewer: ", "")
speak(introDialog)
answer = Start_recording_answer()
feedback = get_feedback(introDialog, answer[0]['DisplayText'])
dialog += [{
    'interviewer': introDialog,
    'interviewee': answer[0]['DisplayText'],
    'feedback': feedback
}]


In [ ]:
for i in range(0, numberOfQuestions):
    prompt = """
        interviewer: {0}

        interviewee: {1}

        interviewer:
        """.format(dialog[len(dialog)-1]['interviewer'], dialog[len(dialog)-1]['interviewee'])
    interviewer = complete_openai(
        prompt=prompt,
        token=600
    )
    print(interviewer[0])
    print(prompt)
    speak(interviewer[0])
    answer = Start_recording_answer()
    feedback = get_feedback(interviewer[0], answer[0]['DisplayText'])
    dialog += [{
        'interviewer': introDialog,
        'interviewee': answer[0]['DisplayText'],
        'feedback': feedback
    }]


In [ ]:
for interaction in dialog:
    speak("""
for the question: {0}

the feedback is: {1}
""".format(interaction['interviewer'],interaction['feedback']))